In [1]:
from torchvision import transforms,datasets
from torch.utils.data import DataLoader
from torch.optim import Adam
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),

    transforms.Normalize(mean=0.1307,std=0.3081)

])
train_dataset = datasets.MNIST(root='../dataset', train=True, download=True, transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = datasets.MNIST(root='../dataset', train=False, download=True, transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)


Failed to download (trying next):
HTTP Error 404: Not Found



100.0%


Extracting ../dataset\MNIST\raw\train-images-idx3-ubyte.gz to ../dataset\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100.0%


Extracting ../dataset\MNIST\raw\train-labels-idx1-ubyte.gz to ../dataset\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100.0%


Extracting ../dataset\MNIST\raw\t10k-images-idx3-ubyte.gz to ../dataset\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100.0%

Extracting ../dataset\MNIST\raw\t10k-labels-idx1-ubyte.gz to ../dataset\MNIST\raw



In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 10)
        self.fc3 = nn.Linear(10, 10)
        self.softmax = nn.LogSoftmax(dim=1)
    def forward(self, x):
        x = x.view(-1,28*28)
        x = self.softmax(self.fc1(x))
        x = self.softmax(self.fc2(x))
        x = self.fc3(x)
        return x

model = Net()
model = model.to(device)

In [4]:
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(),lr=0.001)

In [5]:
def train(num_epochs):
    model.train()
    losses = []
    epoches = torch.arange(1, num_epochs+1)
    for epoch in range(num_epochs):
        running_loss = 0.0
        for idx, (x,y) in enumerate(train_dataloader):
            optimizer.zero_grad()
            x,y = x.to(device), y.to(device)
            y_pred = model(x)
            loss = criterion(y_pred, y)

            loss.backward()
            optimizer.step()
            running_loss += loss.item()*x.size(0)
            if (idx+1) % 100 == 0:
                print(f"Epoch [{epoch+1}/{num_epochs}],Batches[{idx+1}/{len(train_dataloader)}], Loss: {running_loss/((idx+1)*(x.size(0))):.3f}")
        losses.append(running_loss/len(train_dataloader.dataset))
    plt.figure(figsize=(6, 4))
    plt.plot(epoches, losses)
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.show()


def test(model):
    model.eval()  # 设置模型为评估模式
    correct = 0
    total = 0

    with torch.no_grad():  # 不计算梯度
        for images, labels in test_dataloader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)  # 获取预测结果
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Test Accuracy: {100 * correct / total:.2f}%')

In [6]:
if __name__ == '__main__':
    num_epochs = 2
    train(num_epochs)
    test(model)


Epoch [1/2],Batches[100/938], Loss: 1.496
Epoch [1/2],Batches[200/938], Loss: 1.052
Epoch [1/2],Batches[300/938], Loss: 0.857
Epoch [1/2],Batches[400/938], Loss: 0.740


KeyboardInterrupt: 